## 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page <font color='red'> For Seller to update:[Title_of_your_product](Provide link to your marketplace listing of your product).</font>
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

## SNOMED Resolver Pipeline

- **Model**: `snomed_vdb_resolver`
- **Model Description**: This pretrained pipeline extracts clinical entities from clinical text and maps them to their corresponding SNOMED codes.

In [4]:
model_package_arn = "<Customer to specify Model package ARN corresponding to their AWS region>"

In [ ]:
import json
import os
import boto3
import pandas as pd
import sagemaker as sage
from sagemaker import ModelPackage
from sagemaker import get_execution_role
from IPython.display import display
from urllib.parse import urlparse


In [5]:
sagemaker_session = sage.Session()
s3_bucket = sagemaker_session.default_bucket()
region = sagemaker_session.boto_region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")
role = get_execution_role()

sagemaker = boto3.client("sagemaker")
s3_client = sagemaker_session.boto_session.client("s3")
ecr = boto3.client("ecr")
sm_runtime = boto3.client("sagemaker-runtime")

# Set display options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [6]:
model_name = "snomed-vdb-resolver"

real_time_inference_instance_type = "ml.m4.xlarge"
batch_transform_inference_instance_type = "ml.m4.2xlarge"

## 2. Create a deployable model from the model package.

In [7]:
model = ModelPackage(
    role=role, 
    model_package_arn=model_package_arn,
    sagemaker_session=sagemaker_session,
)

### Input Format

To use the model, you need to provide input in one of the following supported formats:

#### JSON Format

Provide input as JSON. We support two variations within this format:

1. **Array of Text Documents**: 
   Use an array containing multiple text documents. Each element represents a separate text document.

   ```json
   {
       "text": [
           "Text document 1",
           "Text document 2",
           ...
       ]
   }

    ```

2. **Single Text Document**:
   Provide a single text document as a string.


   ```json
    {
        "text": "Single text document"
    }
   ```

#### JSON Lines (JSONL) Format

Provide input in JSON Lines format, where each line is a JSON object representing a text document.

```
{"text": "Text document 1"}
{"text": "Text document 2"}
```

## 3. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

### A. Deploy the SageMaker model to an endpoint

In [8]:
predictor = model.deploy(
    initial_instance_count=1,
    instance_type=real_time_inference_instance_type, 
    endpoint_name=model_name,
)

-------------!

Once endpoint has been created, you would be able to perform real-time inference.

In [9]:
def invoke_realtime_endpoint(record, content_type="application/json", accept="application/json"):
    response = sm_runtime.invoke_endpoint(
        EndpointName=model_name,
        ContentType=content_type,
        Accept=accept,
        Body=json.dumps(record) if content_type == "application/json" else record,
    )

    response_body = response["Body"].read().decode("utf-8")

    if accept == "application/json":
        return json.loads(response_body)
    elif accept == "application/jsonlines":
        return response_body
    else:
        raise ValueError(f"Unsupported accept type: {accept}")

### Initial Setup

In [10]:
docs = [
    """The patient is a 40-year-old white male who presents with a chief complaint of "chest pain". The patient is diabetic and has a prior history of coronary artery disease. The patient presents today stating that his chest pain started yesterday evening and has been somewhat intermittent. He has been advised 81 milligrams QDay, 50 units in a.m., 50 mg QDay, 1/150 sublingually PRN chest pain.""",
    
    """Two weeks prior to presentation, she was treated with a five-day course of for a respiratory tract infection. She was on , , and for T2DM and and for HTG. She had been on for six months at the time of presentation. The patient was seen by the endocrinology service and she was discharged on 40 units of at night, 12 units of with meals, and 1000 mg two times a day."""
]

sample_text = """This is an 82-year-old male with a history of prior tobacco use, hypertension, chronic renal insufficiency, chronic obstructive pulmonary disease, gastritis, and transient ischemic attack. He initially presented to Braintree with ST elevation and was transferred to St. Margaret’s Center. He underwent cardiac catheterization because of the left main coronary artery stenosis, which was complicated by hypotension and bradycardia. He required atropine, IV fluids, and dopamine."""

### JSON

In [11]:
input_json_data = {"text": sample_text}
response_json = invoke_realtime_endpoint(input_json_data, content_type="application/json", accept="application/json")
pd.DataFrame(response_json["predictions"][0])

,begin,end,ner_chunk,ner_label,ner_confidence,concept_code,resolution,score,all_codes,domain_class,all_resolutions,all_score
0,52,58,tobacco,Smoking,0.9935,57264008,tobacco,1.000000,"[57264008, 66562002, 102407002, 84498003, 26663004]","[Organism, Substance, Substance, Substance, Substance]","[tobacco, cigarette smoking tobacco, tobacco smoke, pipe smoking tobacco, cigar smoking tobacco]","[1.000000238418579, 0.9128879904747009, 0.9106560945510864, 0.8789343237876892, 0.8733211159706116]"
1,65,76,hypertension,Hypertension,0.9999,38341003,hypertension,1.000000,"[38341003, 75367002, 73578008, 28119000, 38341003]","[Clinical Finding, Observable Entity, Morph Abnormality, Clinical Finding, Clinical Finding]","[hypertension, blood pressure, hyperdistension, renal hypertension, high blood pressure]","[0.9999998807907104, 0.9066827297210693, 0.8890687823295593, 0.8857569694519043, 0.8678926825523376]"
2,79,105,chronic renal insufficiency,Kidney_Disease,0.91976666,723190009,chronic renal insufficiency,1.000000,"[723190009, 425369003, 42399005, 90688005, 42399005]","[Clinical Finding, Clinical Finding, Clinical Finding, Clinical Finding, Clinical Finding]","[chronic renal insufficiency, chronic progressive renal insufficiency, renal insufficiency, chronic renal failure, renal insufficiency syndrome]","[1.000000238418579, 0.9239168167114258, 0.9123026728630066, 0.8822140693664551, 0.8708065748214722]"
3,108,144,chronic obstructive pulmonary disease,Disease_Syndrome_Disorder,0.5127,13645005,chronic obstructive pulmonary disease,1.000000,"[13645005, 13645005, 13645005, 13645005, 13645005]","[Clinical Finding, Clinical Finding, Clinical Finding, Clinical Finding, Clinical Finding]","[chronic obstructive pulmonary disease, chronic obstructive lung disease, copd - chronic obstructive pulmonary disease, chronic obstructive airway disease, copd]","[1.0000001192092896, 0.953446626663208, 0.9277710914611816, 0.8972681760787964, 0.8662362694740295]"
4,147,155,gastritis,Disease_Syndrome_Disorder,0.9959,4556007,gastritis,1.000000,"[4556007, 723096000, 25458004, 1824008, 57433008]","[Clinical Finding, Clinical Finding, Clinical Finding, Clinical Finding, Clinical Finding]","[gastritis, bacterial gastritis, acute gastritis, allergic gastritis, reflux gastritis]","[1.000000238418579, 0.904236912727356, 0.8971583247184753, 0.8965718746185303, 0.8924410343170166]"
5,162,186,transient ischemic attack,Cerebrovascular_Disease,0.5781,266257000,transient ischemic attack,1.000000,"[266257000, 266257000, 57357009, 266257000, 266257000]","[Clinical Finding, Clinical Finding, Clinical Finding, Clinical Finding, Clinical Finding]","[transient ischemic attack, transient ischaemic attack, transient ischemia, tia - transient ischemic attack, transient cerebral ischemia]","[0.9999999403953552, 0.9563196897506714, 0.952727198600769, 0.9329274296760559, 0.928642988204956]"
6,230,241,ST elevation,snomed_term,0.64225,164931005,st elevation,1.000000,"[164931005, 76388001, 255456001, 103720008, 705008001]","[Observable Entity, Clinical Finding, Qualifier Value, Procedure, Clinical Finding]","[st elevation, st segment elevation, elevation, elevation, nos, anterior st segment elevation]","[1.0000004768371582, 0.8871583938598633, 0.8315354585647583, 0.8132315874099731, 0.8127869963645935]"
7,302,324,cardiac catheterization,Procedure,0.7781,41976001,cardiac catheterization,1.000000,"[41976001, 41976001, 41976001, 45211000, 41976001]","[Procedure, Procedure, Procedure, Procedure, Procedure]","[cardiac catheterization, cardiac catheterization procedure, cardiac catheterisation, catheterization, cardiac catheterisation procedure]","[1.0000001192092896, 0.9511892199516296, 0.9394111037254333, 0.9253654479980469, 0.9106451272964478]"
8,337,374,the left main coronary artery stenosis,PROBLEM,0.84225005,371804009,left main coronary artery disease,0.916886,"[371804009, 876857001, 1255569006, 1255569006, 1255569006]","[Clinical Finding, Clinical Finding, No_Concept_Class, No_Concept_Cla

### JSON Lines

In [13]:
def create_jsonl(records):
    if isinstance(records, str):
        records = [records]
    json_records = [{"text": text} for text in records]
    json_lines = "\n".join(json.dumps(record) for record in json_records)
    return json_lines

In [14]:
input_jsonl_data = create_jsonl(sample_text)
data = invoke_realtime_endpoint(input_jsonl_data, content_type="application/jsonlines" , accept="application/jsonlines" )
print(data)

{"predictions": [{"begin": 52, "end": 58, "ner_chunk": "tobacco", "ner_label": "Smoking", "ner_confidence": "0.9935", "concept_code": "57264008", "resolution": "tobacco", "score": 1.000000238418579, "all_codes": ["57264008", "66562002", "102407002", "84498003", "26663004"], "domain_class": ["Organism", "Substance", "Substance", "Substance", "Substance"], "all_resolutions": ["tobacco", "cigarette smoking tobacco", "tobacco smoke", "pipe smoking tobacco", "cigar smoking tobacco"], "all_score": [1.000000238418579, 0.9128879904747009, 0.9106560945510864, 0.8789343237876892, 0.8733211159706116]}, {"begin": 65, "end": 76, "ner_chunk": "hypertension", "ner_label": "Hypertension", "ner_confidence": "0.9999", "concept_code": "38341003", "resolution": "hypertension", "score": 0.9999998807907104, "all_codes": ["38341003", "75367002", "73578008", "28119000", "38341003"], "domain_class": ["Clinical Finding", "Observable Entity", "Morph Abnormality", "Clinical Finding", "Clinical Finding"], "all_res

### B. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [16]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

## 4. Batch inference

In [17]:
validation_json_file_name = "input.json"
validation_jsonl_file_name = "input.jsonl"

validation_input_json_path = f"s3://{s3_bucket}/{model_name}/validation-input/json/"
validation_output_json_path = f"s3://{s3_bucket}/{model_name}/validation-output/json/"

validation_input_jsonl_path = f"s3://{s3_bucket}/{model_name}/validation-input/jsonl/"
validation_output_jsonl_path = f"s3://{s3_bucket}/{model_name}/validation-output/jsonl/"

def upload_to_s3(input_data, file_name):
    file_format = os.path.splitext(file_name)[1].lower()
    s3_client.put_object(
        Bucket=s3_bucket,
        Key=f"{model_name}/validation-input/{file_format[1:]}/{file_name}",
        Body=input_data.encode("UTF-8"),
    )

In [18]:
# Create JSON and JSON Lines data
input_jsonl_data = create_jsonl(docs)
input_json_data = json.dumps({"text": docs})

# Upload JSON and JSON Lines data to S3
upload_to_s3(input_json_data, validation_json_file_name)
upload_to_s3(input_jsonl_data, validation_jsonl_file_name)

### JSON

In [ ]:
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/json",
    output_path=validation_output_json_path
)

transformer.transform(validation_input_json_path, content_type="application/json")
transformer.wait()

In [ ]:
def retrieve_json_output_from_s3(validation_file_name):
    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = json.loads(response["Body"].read().decode("utf-8"))
    display(data)

In [22]:
retrieve_json_output_from_s3(validation_json_file_name)

{'predictions': [[{'begin': 80,
    'end': 89,
    'ner_chunk': 'chest pain',
    'ner_label': 'Symptom',
    'ner_confidence': '0.7448',
    'concept_code': '29857009',
    'resolution': 'chest pain',
    'score': 1.0,
    'all_codes': ['29857009',
     '426396005',
     '274664007',
     '279084009',
     '2237002'],
    'domain_class': ['Clinical Finding',
     'Clinical Finding',
     'Clinical Finding',
     'Clinical Finding',
     'Clinical Finding'],
    'all_resolutions': ['chest pain',
     'cardiac chest pain',
     'chest pain on breathing',
     'chest discomfort',
     'pleuritic chest pain'],
    'all_score': [1.0,
     0.9261186122894287,
     0.8930981159210205,
     0.889647364616394,
     0.8866108655929565]},
   {'begin': 108,
    'end': 115,
    'ner_chunk': 'diabetic',
    'ner_label': 'Diabetes',
    'ner_confidence': '0.9994',
    'concept_code': '280137006',
    'resolution': 'diabetic foot',
    'score': 0.8424344062805176,
    'all_codes': ['280137006',
     

### JSON Lines

In [ ]:
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/jsonlines",
    output_path=validation_output_jsonl_path
)
transformer.transform(validation_input_jsonl_path, content_type="application/jsonlines")
transformer.wait()

In [ ]:
def retrieve_jsonlines_output_from_s3(validation_file_name):

    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = response["Body"].read().decode("utf-8")
    print(data)

In [25]:
retrieve_jsonlines_output_from_s3(validation_jsonl_file_name)

{"predictions": [{"begin": 80, "end": 89, "ner_chunk": "chest pain", "ner_label": "Symptom", "ner_confidence": "0.7448", "concept_code": "29857009", "resolution": "chest pain", "score": 1.0, "all_codes": ["29857009", "426396005", "274664007", "279084009", "2237002"], "domain_class": ["Clinical Finding", "Clinical Finding", "Clinical Finding", "Clinical Finding", "Clinical Finding"], "all_resolutions": ["chest pain", "cardiac chest pain", "chest pain on breathing", "chest discomfort", "pleuritic chest pain"], "all_score": [1.0, 0.9261186122894287, 0.8930981159210205, 0.889647364616394, 0.8866108655929565]}, {"begin": 108, "end": 115, "ner_chunk": "diabetic", "ner_label": "Diabetes", "ner_confidence": "0.9994", "concept_code": "280137006", "resolution": "diabetic foot", "score": 0.8424344062805176, "all_codes": ["280137006", "160670007", "385804009", "309350005", "170747006"], "domain_class": ["Clinical Finding", "Clinical Finding", "Procedure", "Social Context", "Clinical Finding"], "al

In [26]:
model.delete_model()

INFO:sagemaker:Deleting model with name: snomed-vdb-resolver-2025-07-31-12-09-13-530


### Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

